## LFR benchmark for Altmap vs Map Eq
### Compare altmap to map eq using networkx

In [6]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.pyplot import Line2D
import pandas as pd
import numpy as np
from collections import OrderedDict

# show plots in separate window
%pylab
# load helpers and wrappers
%run helpers.py 

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [7]:
from networkx.generators.community import LFR_benchmark_graph
from sklearn.metrics import normalized_mutual_info_score as nmi_score
from sklearn.metrics import adjusted_mutual_info_score as ami_score

# generate LFR benchmark graph + extract ground truth communities
def generate_LFR_benchmark(N = 500, mu = 0.1):
    
    # LFR params N=10000
    # params = {'max_degree':50, 'max_community':100, 'min_community':50, 'average_degree':20, 'tau1':3.0, 'tau2':1.5}
    
    # LFR params generic
    max_community = int(0.2*N)
    min_community = int(max_community * 0.25) #0.5
    max_degree = int(max_community * 0.3) # 0.6
    min_degree = int(min_community * 0.4)
    gamma = 3.5 # Power law exponent for the degree distribution 
    beta = 1.1 #3.0 # Power law exponent for the community size distribution
    params = {'max_degree':max_degree, 'max_community':max_community, 'min_community':min_community,
              'min_degree':min_degree, 'tau1':gamma, 'tau2':beta}
    

    max_degree = params['max_degree']
    max_community = params['max_community']
    min_community = params['min_community']
    #average_degree = params['average_degree']
    min_degree = params['min_degree']
    tau1 = params['tau1'] # Power law exponent for the degree distribution 
    tau2 = params['tau2'] # Power law exponent for the community size distribution

    # generate LFR benchmark graph
    
    G = LFR_benchmark_graph(N, tau1, tau2, mu, min_degree=min_degree, max_degree=max_degree, 
                            max_community=max_community, min_community=min_community)
    G = nx.convert_node_labels_to_integers(G, first_label=1)
    
    # extract ground truth communities from networkx graph object
    communities_true = {}
    num_communities = 0
    for n in range(1,N+1):
        if n in communities_true:
            continue
            
        num_communities = num_communities + 1
        community = G.nodes[n]['community']
        node_ids = np.asarray(list(community))
        node_ids = node_ids + 1 # have node labels >= 1
        communities_true.update(dict.fromkeys(node_ids , num_communities))
        
    communities_true = OrderedDict(sorted(communities_true.items()))
    num_communities_true = max(communities_true.values()) - min(communities_true.values()) + 1
    
    return G, communities_true, num_communities_true

# compute normalized mutual information between two partitions
def compute_score(communities_true, communities_found):
    labels_true = list(communities_true.values())
    labels_found = list(communities_found.values())

    # return nmi_score(labels_true,labels_found, average_method='arithmetic')
    return ami_score(labels_true,labels_found, average_method='arithmetic')

class BenchmarkResults:
    def __init__(self, var_list, num_realizations):
        self.var_list = var_list
        self.num_datapoints = len(var_list)
        self.num_realizations = num_realizations
        self.actual_realizations = np.zeros((self.num_datapoints,), dtype=int)
        self.scores = np.zeros((num_realizations, self.num_datapoints))
        self.errors = np.zeros((num_realizations, self.num_datapoints))
    
    def evaluate_results(self, use_median=False):
        self.mean_scores = np.empty((self.num_datapoints,))
        self.mean_scores[:] = np.nan
        self.std_scores = np.empty((self.num_datapoints,))
        self.std_scores[:] = np.nan
        self.mean_errors = np.empty((self.num_datapoints,))
        self.mean_errors[:] = np.nan
        self.std_errors = np.empty((self.num_datapoints,))
        self.std_errors[:] = np.nan
        
        for i in range(self.num_datapoints):
            nr = self.actual_realizations[i]
            if nr == 0:
                continue
            
            self.std_scores[i] = np.std(self.scores[:nr, i], ddof=1)
            self.std_errors[i] = np.std(self.errors[:nr, i], ddof=1)
            if (use_median):
                self.mean_scores[i] = np.median(self.scores[:nr, i])
                self.mean_errors[i] = np.median(self.errors[:nr, i])
            else:
                self.mean_scores[i] = np.mean(self.scores[:nr, i])
                self.mean_errors[i] = np.mean(self.errors[:nr, i])
    
    def write_csv(self, path):
        self.evaluate_results() # make sure we have sth to write
        
        df = pd.DataFrame()
        df['var_list'] = self.var_list
        df['actual_realizations'] = self.actual_realizations
        df['mean_scores'] = self.mean_scores
        df['std_scores'] = self.std_scores
        df['mean_errors'] = self.mean_errors
        df['std_errors'] = self.std_errors
        
        df.to_csv(path, index_label='id')
    
    @classmethod
    def read_csv(cls, path):
        df = pd.read_csv(path, index_col=0)
        
        num_realizations = int(np.max(df['actual_realizations'].values))
        results = cls(df['var_list'].values, num_realizations)
        
        results.actual_realizations = df['actual_realizations'].values
        results.mean_scores = df['mean_scores'].values
        results.std_scores  = df['std_scores'].values
        results.mean_errors = df['mean_errors'].values
        results.std_errors  = df['std_errors'].values
        
        return results
            
            
        
# LFR Benchmark
# num_realizations .. number of network realizations for each parameter pair (mu, N)
def run_benchmark(N_list: list, mu_list: list, num_realizations=10):
    
    N = N_list[0]
    var_list = mu_list
    vary_mu = True
    if len(N_list) > 1:
        mu = mu_list[0]
        var_list = N_list
        vary_mu = False
    
    num_benchmarks = len(var_list) * num_realizations
    benchmark_id = 0
           
    infomap_results = BenchmarkResults(var_list, num_realizations)
    altmap_results = BenchmarkResults(var_list, num_realizations)
    altmap_sci_results = BenchmarkResults(var_list, num_realizations)
    sci_results = BenchmarkResults(var_list, num_realizations)
    for var_idx, var in enumerate(var_list):
        if vary_mu:
            mu = var
        else:
            N = var
            
        realization_idx = -1
        for _ in range(num_realizations):
            benchmark_id = benchmark_id + 1
            print(f'Starting benchmark {benchmark_id}/{num_benchmarks} for (N,mu) = ({N},{mu})\n')
            try:
                G, communities_true, num_communities_true = generate_LFR_benchmark(N, mu)
            except nx.ExceededMaxIterations as err:
                print(f'Failed to generate network for (N,mu) = ({N},{mu}): ', err)
                continue
            
            realization_idx += 1
            
            # test infomap
            communities_found, num_communities_found, _, _ = infomap(G, altmap=False)
            print (f'Infomap found {num_communities_found} communities vs. {num_communities_true} ground truth '
                   f'communities.\n')
            
            score = compute_score(communities_true, communities_found)
            infomap_results.scores[realization_idx, var_idx] = score
            error = num_communities_found/num_communities_true - 1.0
            infomap_results.errors[realization_idx, var_idx] = error
            
            # test altmap
            communities_found, num_communities_found, _, _ = infomap(G, altmap=True, update_inputfile=False)
            print (f'Altmap found {num_communities_found} communities vs. {num_communities_true} ground truth '
                   f'communities.\n')
            
            score = compute_score(communities_true, communities_found)
            altmap_results.scores[realization_idx, var_idx] = score
            error = num_communities_found/num_communities_true - 1.0
            altmap_results.errors[realization_idx, var_idx] = error
        
            # test altmap with SCI
            communities_found, num_communities_found,\
            communities_init, num_communities_init = infomap(G, altmap=True, init='sc', update_inputfile=False)
            print (f'Altmap with SCI ({num_communities_init}) found {num_communities_found} communities vs. '
                   f'{num_communities_true} '
                   f'ground truth communities.\n')
            
            score = compute_score(communities_true, communities_found)
            altmap_sci_results.scores[realization_idx, var_idx] = score
            error = num_communities_found/num_communities_true - 1.0
            altmap_sci_results.errors[realization_idx, var_idx] = error
            
            score = compute_score(communities_true, communities_init)
            sci_results.scores[realization_idx, var_idx] = score
            error = num_communities_init/num_communities_true - 1.0
            sci_results.errors[realization_idx, var_idx] = error
        
        # store actual number of realizations    
        infomap_results.actual_realizations[var_idx] = realization_idx + 1
        altmap_results.actual_realizations[var_idx] = realization_idx + 1
        altmap_sci_results.actual_realizations[var_idx] = realization_idx + 1
        sci_results.actual_realizations[var_idx] = realization_idx + 1

    print(f'Finished benchmark successfully!\n')
    return infomap_results, altmap_results, altmap_sci_results, sci_results

def plot_benchmark_results(results: BenchmarkResults, type='scores', color='blue', marker=None, label=None, 
                           lower_bound=None, upper_bound=None, plot_uncertainty=True):
            
    xdata = results.var_list
    data = results.mean_scores
    data_std = results.std_scores
    
    if type == 'errors':
        data = results.mean_errors
        data_std = results.std_errors
        
    lw = 2; ms = 10
    line = plt.plot(xdata, data, '--', marker = marker, color=color, linewidth=lw, markersize=ms, label=label)
    
    if plot_uncertainty:
        upper = data + data_std
        lower = data - data_std
        if lower_bound != None:
            lower[lower < lower_bound] = lower_bound
        if upper_bound != None:
            upper[upper > upper_bound] = upper_bound
            
        plt.fill_between(xdata, upper, lower, color=color, alpha=0.25)
    
    return line

In [70]:
import warnings
warnings.filterwarnings('ignore')

# benchmark params
# N_list = [200]
# mu_list = np.linspace(0.15, 0.75, 10)
N_list = [200, 300, 400, 500, 700, 900, 1200, 1500, 2000]
mu_list = [0.5]
num_realizations = 10

infomap_results, altmap_results, altmap_sci_results, sci_results = run_benchmark(N_list, mu_list, num_realizations)

Starting benchmark 1/90 for (N,mu) = (200,0.5)

Infomap found 14 communities vs. 13 ground truth communities.

Altmap found 29 communities vs. 13 ground truth communities.

Spectral clustering finished in 0.1528360000002067 seconds.
Altmap with SCI (14) found 16 communities vs. 13 ground truth communities.

Starting benchmark 2/90 for (N,mu) = (200,0.5)

Infomap found 15 communities vs. 8 ground truth communities.

Altmap found 33 communities vs. 8 ground truth communities.

Spectral clustering finished in 0.14033599999993385 seconds.
Altmap with SCI (14) found 18 communities vs. 8 ground truth communities.

Starting benchmark 3/90 for (N,mu) = (200,0.5)

Infomap found 12 communities vs. 9 ground truth communities.

Altmap found 28 communities vs. 9 ground truth communities.

Spectral clustering finished in 0.13568499999973938 seconds.
Altmap with SCI (14) found 16 communities vs. 9 ground truth communities.

Starting benchmark 4/90 for (N,mu) = (200,0.5)

Infomap found 16 communities 

In [71]:
infomap_results.evaluate_results()
altmap_results.evaluate_results()
altmap_sci_results.evaluate_results()
sci_results.evaluate_results()

print (f'Minimum number of realizations is {np.min(infomap_results.actual_realizations)}.')
print (f'Actual realizations are {infomap_results.actual_realizations}.')
print (f'Average realizations are {np.mean(infomap_results.actual_realizations)}.')

Minimum number of realizations is 9.
Actual realizations are [10 10 10 10 10 10 10 10  9].
Average realizations are 9.88888888888889.


In [72]:
bench_type_str = f'{N_list[0]}n'
if len(N_list) > 1:
    bench_type_str = f'{int(mu_list[0] * 100)}mu'

infomap_results.write_csv(f'./lfr_results/infomap_' + bench_type_str + '.csv')
altmap_results.write_csv(f'./lfr_results/altmap_' + bench_type_str + '.csv')
altmap_sci_results.write_csv(f'./lfr_results/altmap_sci_' + bench_type_str + '.csv')
sci_results.write_csv(f'./lfr_results/sci_' + bench_type_str + '.csv')


In [9]:
# bench_type_str = '2000n'
bench_type_str = '35mu'
acc_results = BenchmarkResults.read_csv(f'./lfr_results/acc_' + bench_type_str + '.csv')
infomap_results = BenchmarkResults.read_csv(f'./lfr_results/infomap_' + bench_type_str + '.csv')
altmap_results = BenchmarkResults.read_csv(f'./lfr_results/altmap_' + bench_type_str + '.csv')
altmap_sci_results = BenchmarkResults.read_csv(f'./lfr_results/altmap_sci_' + bench_type_str + '.csv')
sci_results = BenchmarkResults.read_csv(f'./lfr_results/sci_' + bench_type_str + '.csv')


In [61]:
plt.close('all')
fig, ax = plt.subplots(figsize=(12,9))
# plt.title(f'LFR benchmark, N = {N} nodes')
plt.plot([0.5, 0.5], [0,1], 'r')

plot_benchmark_results(infomap_results, color='royalblue', marker='x', label='Infomap', lower_bound=0.0, 
                       upper_bound=1.0)
# plot_benchmark_results(sci_results, color='crimson', marker='s', label='SCI')
plot_benchmark_results(altmap_results, color='darkorange', marker='^', label='Synthesizing Infomap')
plot_benchmark_results(altmap_sci_results, color='seagreen', marker='o', label='Synthesizing Infomap with SCI')

plt.grid()
plt.xlabel('Mixing parameter $\mu$')
plt.ylabel(r'$\mathrm{AMI}(\mathcal{Y},\mathcal{Y}_{true})$')
plt.legend()
plt.tight_layout()

In [7]:
min_err = np.min(np.min([infomap_results.mean_errors - infomap_results.std_errors,\
                         altmap_results.mean_errors - altmap_results.std_errors,\
                         altmap_sci_results.mean_errors - altmap_sci_results.std_errors]))

max_err = np.max(np.max([infomap_results.mean_errors + infomap_results.std_errors,\
                         altmap_results.mean_errors + altmap_results.std_errors,\
                         altmap_sci_results.mean_errors + altmap_sci_results.std_errors]))
                                                  
plt.close('all')
fig, ax = plt.subplots(figsize=(12,9))
# plt.title(f'LFR benchmark, N = {N} nodes')
plt.plot([0.5, 0.5], [min_err-0.2, max_err+0.2], 'r')

plot_benchmark_results(infomap_results, type='errors', color='royalblue', marker='x', label='Infomap', lower_bound=-1.0)
# plot_benchmark_results(sci_results, type='errors', color='crimson', marker='s', label='SCI')
plot_benchmark_results(altmap_results, type='errors', color='darkorange', marker='^', label='Synthesizing Infomap')
plot_benchmark_results(altmap_sci_results, type='errors', color='seagreen', marker='o', label='Synthesizing Infomap '
                                                                                              'with SCI')
plt.grid()
plt.xlabel('Mixing parameter $\mu$')
plt.ylabel(r'Mean relative error $\bar{e}_\theta$')
plt.legend(loc='upper left')
plt.tight_layout()

In [17]:
plt.close('all')
fig, ax = plt.subplots(figsize=(12,9))

ln1=plot_benchmark_results(infomap_results, color='royalblue', marker='x', label='Infomap', lower_bound=0.0, 
                       upper_bound=1.0)
ln2=plot_benchmark_results(altmap_results, color='darkorange', marker='^', label='Synthesizing Infomap')
ln3=plot_benchmark_results(altmap_sci_results, color='seagreen', marker='o', label='Synthesizing Infomap with SCI')
ax_acc = ax.twinx()
ln4=plot_benchmark_results(acc_results, color='crimson', marker='s', label='Average Community Clustering', 
                           plot_uncertainty=False)

plt.grid()
ax.set_xlabel('Number of nodes $N$')
ax.set_ylabel(r'$\mathrm{AMI}(\mathcal{Y},\mathcal{Y}_{true})$')
ax_acc.set_ylabel(r'$\mathrm{ACC}(\mathcal{Y}_{true})$', color='crimson')
ax_acc.tick_params(axis='y', labelcolor='crimson')
leg = ln1 + ln2 + ln3 + ln4
labs = [l.get_label() for l in leg]
ax.legend(leg, labs, loc=5)
plt.tight_layout()

In [13]:
min_err = np.min(np.min([infomap_results.mean_errors - infomap_results.std_errors,\
                         altmap_results.mean_errors - altmap_results.std_errors,\
                         altmap_sci_results.mean_errors - altmap_sci_results.std_errors]))

max_err = np.max(np.max([infomap_results.mean_errors + infomap_results.std_errors,\
                         altmap_results.mean_errors + altmap_results.std_errors,\
                         altmap_sci_results.mean_errors + altmap_sci_results.std_errors]))
                                                  
plt.close('all')
fig, ax = plt.subplots(figsize=(12,9))

plot_benchmark_results(infomap_results, type='errors', color='royalblue', marker='x', label='Infomap', lower_bound=-1.0)
# plot_benchmark_results(sci_results, type='errors', color='crimson', marker='s', label='SCI')
plot_benchmark_results(altmap_results, type='errors', color='darkorange', marker='^', label='Synthesizing Infomap')
plot_benchmark_results(altmap_sci_results, type='errors', color='seagreen', marker='o', label='Synthesizing Infomap '
                                                                                              'with SCI')
plt.grid()
plt.xlabel('Number of nodes $N$')
plt.ylabel(r'Mean relative error $\bar{e}_\theta$')
plt.legend(loc='upper right')
plt.tight_layout()

In [28]:
# LFR params
N = 5000
mu = 0.01
max_community = int(0.5*N)
min_community = int(max_community * 0.25)
max_degree = int(max_community * 0.3)
min_degree = int(min_community * 0.4)
gamma = 3.5 # Power law exponent for the degree distribution 
beta = 1.1 # Power law exponent for the community size distribution
max_iter = 500

# generate LFR benchmark graph
G = LFR_benchmark_graph(N, gamma, beta, mu, min_degree=min_degree, max_degree=max_degree, 
                        max_community=max_community, min_community=min_community, max_iters=max_iter, tol=1)



In [14]:
def avg_community_clusterings(G, communities):
    _, comm_nodes,_ = nodes_per_community(communities)
    num_comms = len(comm_nodes)
    comm_clusterings = []
    for i in range(num_comms):
        tmp_list = comm_nodes.copy()
        tmp_list.pop(i)
        tmp_list = [node for comm in tmp_list for node in comm]
    
        G_tmp = G.copy()
        G_tmp.remove_nodes_from(tmp_list)
        comm_clusterings.append(nx.average_clustering(G_tmp))
        
    return np.mean(comm_clusterings)

N_list = [200, 300, 400, 500, 700, 900, 1200, 1500, 2000]
mu = 0.5
num_realizations = 20
num_benchmarks = num_realizations * len(N_list)
acc_results = BenchmarkResults(N_list, num_realizations)

benchmark_id = 0
acc_list = []
for N_idx, N in enumerate(N_list):
    cc_list = []
    realization_idx = -1
    for _ in range(num_realizations):
        benchmark_id = benchmark_id + 1
        print(f'Starting benchmark {benchmark_id}/{num_benchmarks} for (N,mu) = ({N},{mu})\n')
        try:
            G, communities_true,_ = generate_LFR_benchmark(N, mu)
        except nx.ExceededMaxIterations as err:
            print(f'Failed to generate network for (N,mu) = ({N},{mu}): ', err)
            continue
        
        realization_idx += 1
        
        # compute acc
        score = avg_community_clusterings(G, communities_true)
        acc_results.scores[realization_idx, N_idx] = score
                    
    # store actual number of realizations
    acc_results.actual_realizations[N_idx] = realization_idx + 1

print(f'Finished benchmark successfully!\n')   

Starting benchmark 1/180 for (N,mu) = (200,0.5)

Starting benchmark 2/180 for (N,mu) = (200,0.5)

Starting benchmark 3/180 for (N,mu) = (200,0.5)

Starting benchmark 4/180 for (N,mu) = (200,0.5)

Starting benchmark 5/180 for (N,mu) = (200,0.5)

Starting benchmark 6/180 for (N,mu) = (200,0.5)

Starting benchmark 7/180 for (N,mu) = (200,0.5)

Starting benchmark 8/180 for (N,mu) = (200,0.5)

Starting benchmark 9/180 for (N,mu) = (200,0.5)

Starting benchmark 10/180 for (N,mu) = (200,0.5)

Starting benchmark 11/180 for (N,mu) = (200,0.5)

Starting benchmark 12/180 for (N,mu) = (200,0.5)

Starting benchmark 13/180 for (N,mu) = (200,0.5)

Starting benchmark 14/180 for (N,mu) = (200,0.5)

Starting benchmark 15/180 for (N,mu) = (200,0.5)

Starting benchmark 16/180 for (N,mu) = (200,0.5)

Starting benchmark 17/180 for (N,mu) = (200,0.5)

Starting benchmark 18/180 for (N,mu) = (200,0.5)

Starting benchmark 19/180 for (N,mu) = (200,0.5)

Starting benchmark 20/180 for (N,mu) = (200,0.5)

Starting 

In [15]:
acc_results.write_csv(f'./lfr_results/acc_{int(mu*100)}mu.csv')

In [20]:
acc_results.evaluate_results(use_median=False)
print (f'Minimum number of realizations is {np.min(acc_results.actual_realizations)}.')
print (f'Actual realizations are {acc_results.actual_realizations}.')
print (f'Average realizations are {np.mean(acc_results.actual_realizations)}.')

plt.close('all')
fig, ax = plt.subplots(figsize=(12,9))

plot_benchmark_results(acc_results, color='crimson', marker='s', label='Average Community Clustering', )
plt.grid()
plt.xlabel('Number of nodes $N$')
plt.ylabel(r'$\mathrm{ACC}(\mathcal{Y}_{true})$')
plt.legend(loc='upper right')
plt.tight_layout()



Minimum number of realizations is 19.
Actual realizations are [20 20 20 20 20 19 20 20 20].
Average realizations are 19.88888888888889.


In [58]:
G, communities_true, num_communities_true = generate_LFR_benchmark(N=200, mu=0.35)
_, _, comm_sizes = nodes_per_community(communities_true)
print (f'There are {num_communities_true} ground truth communities.')
print (f'Community sizes are {comm_sizes}')

communities_infomap, _, _, _ = infomap(G, altmap=False)
communities_altmap, _, _, _ = infomap(G, altmap=True, update_inputfile=False)
communities_altmap_sci, _, communities_sci, _ = infomap(G, altmap=True, init='sc', update_inputfile=False)
#             
# print(f'Altmap cost for true partition is {altmap_cost(G, communities_true)}')

There are 9 ground truth communities.
Community sizes are [21 31 39 57 53 25 20 16 38]
Spectral clustering finished in 0.17582600000002913 seconds.


In [59]:
def relabel_communities_by_largest(communities):
    comm_ids, comm_nodes, comm_sizes = nodes_per_community(communities)
    tmp = sorted(zip(comm_sizes, comm_ids, comm_nodes), reverse = True)
    sorted_nodes = [node for _,_,nodes in tmp for node in nodes]
    sorted_comm_sizes = [comm_size for comm_size,_,_ in tmp]
    sorted_comms = [i for i, comm_size in enumerate(sorted_comm_sizes) for j in range(comm_size)]
    relabeled_communities = OrderedDict(zip(sorted_nodes, sorted_comms))
    return sorted_nodes, sorted_comms, relabeled_communities

def plot_communities(communities, pos, num_communities_true, aggregate_comms=True):
    # plotting params
    node_size = 250
    edge_width = 0.2
    font_size = 13
    cmap = plt.get_cmap('Set3') # 'Set3' 'jet' 'hsv' commColors
    
    # plot communities
    fig, ax = plt.subplots(figsize=(14,9))
    ax.set_frame_on(False)
    ax.set_yticklabels([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_xticks([])
    # ax.set_title('Ground truth')
    # nx.draw_networkx_edges(G, pos, ax=ax, width=edge_width)
    
    num_colors_used = min(num_communities_true + 1, get_num_communities(communities))
    _,_, relabeled_comms = relabel_communities_by_largest(communities)
    comm_ids, comm_nodes, comm_sizes = nodes_per_community(relabeled_comms)
    tmp = sorted(zip(comm_sizes, comm_ids, comm_nodes), reverse = True)
    # print each community seperately with its own color
    for _, comm_id, nodes in tmp:
        c_idx = comm_id if comm_id < num_colors_used or not aggregate_comms else num_colors_used - 1
        nx.draw_networkx_nodes(G, pos=pos, node_color=cmap(c_idx), cmap=cmap, ax=ax, node_size=node_size, 
                               nodelist=nodes)
    # nx.draw_networkx_labels(G, pos, ax=ax, labels=communities, font_weight='bold', font_size=font_size)
    
    comm_sizes_aggregated = comm_sizes
    if aggregate_comms == True:
        comm_sizes_aggregated = sorted(comm_sizes, reverse=True)
        comm_sizes_aggregated[num_colors_used-1] = sum(comm_sizes_aggregated[num_colors_used-1:])
        comm_sizes_aggregated = comm_sizes_aggregated[:num_colors_used]

    # assemble custom legend for community colors
    custom_lines = [Line2D([], [], color=cmap(c), marker='o', ms=15, lw=0) for c in range(num_colors_used)]
    custom_labels = [f'Community {id} ({size} nodes)' for id, size in enumerate(comm_sizes_aggregated)]
    
    if aggregate_comms == True:
        custom_labels[-1] = f'Residual nodes ({comm_sizes_aggregated[-1]})'
    
    if num_colors_used >= 4:
        res = num_colors_used % 4
        split_size = int(num_colors_used / 4)
        
        start = 0; end = split_size + int(res > 0)
        slc = slice(start, end)
        print (slc)
        leg3 = ax.legend(custom_lines[slc], custom_labels[slc], loc=2, fontsize='small')
        
        start += split_size + int(res > 0); end += split_size + int(res > 1)
        slc = slice(start, end)
        print (slc)
        leg2 = ax.legend(custom_lines[slc], custom_labels[slc], loc=1, fontsize='small')
        
        start += split_size + int(res > 1); end += split_size + int(res > 2)
        slc = slice(start, end)
        print (slc)
        leg1 = ax.legend(custom_lines[slc], custom_labels[slc], loc=3, fontsize='small')
        
        start += split_size + int(res > 2);
        slc = slice(start, None)
        print (slc)
        ax.legend(custom_lines[slc], custom_labels[slc], loc=4, fontsize='small')
        ax.add_artist(leg1)
        ax.add_artist(leg2)
        ax.add_artist(leg3)
    else:
        ax.legend(custom_lines, custom_labels, loc=2, fontsize='small')
    
    plt.tight_layout()
        

plt.close('all')
pos = community_layout(communities_true)
plot_communities(communities_true, pos, num_communities_true, aggregate_comms=False)
plot_communities(communities_infomap, pos, num_communities_true, aggregate_comms=False)
plot_communities(communities_altmap, pos, num_communities_true)
plot_communities(communities_altmap_sci, pos, num_communities_true)

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

slice(0, 3, None)
slice(3, 5, None)
slice(5, 7, None)
slice(7, None, None)
slice(0, 3, None)
slice(3, 5, None)
slice(5, 7, None)
slice(7, None, None)
slice(0, 3, None)
slice(3, 6, None)
slice(6, 8, None)
slice(8, None, None)
slice(0, 3, None)
slice(3, 6, None)
slice(6, 8, None)
slice(8, None, None)
